**Load Required Libraries**

In [ ]:
import cv2
from pytube import YouTube 
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import ffmpeg
import torch
import os
import math
import yaml
from IPython.core.magic import register_line_cell_magic
import pandas as pd
import datetime
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Load Dataset**

In [ ]:
# Load the csv file with YouTube links for Synthetic videos
df_videos_url = pd.read_csv("/content/drive/MyDrive/Thesis/Dataset/SyntheticVideoLinks.csv", sep=",")

In [ ]:
# Load the csv file with YouTube links for real-time videos
df_real_url = pd.read_csv("/content/drive/MyDrive/Thesis/Dataset/RealScenariosVideoLinks.csv", sep=",")

**Split the synthetic videos dataset into train, validation and test data**

In [ ]:
# Split data into train, validation and test sets
def split_dataset(df_dataset):

  # use the train test split library from sklearn to split data into train and test set.
  train_data, test_data = train_test_split(df_dataset, test_size = 0.40, shuffle = True, random_state = 100)
  
  # use the test set to further split the data into test and validation set.
  test_data, validation_data = train_test_split(test_data, test_size = 0.20, random_state = 200)

  return train_data, validation_data, test_data

In [ ]:
# Split data into train test and validation sets
train_videos, validation_videos, test_videos = split_dataset(df_videos_url)

**Download the YouTube videos**

In [ ]:
def download_youtube_videos(dataset_url,base_download_path, setType):

  count = 1  
  for url in dataset_url["Url"]:
    time.sleep(2)   
    youtube_obj = YouTube(url)  
    try:
      youtube_obj.streams.filter(progressive = True, 
                                 file_extension = "mp4").first().download(output_path = base_download_path + setType, 
                                                                          filename = f"VideoInput{setType}{count}.mp4")
      count = count + 1    
    except:
        print("Error downloading video: ", url)
  
  print("Completed - Videos downloaded for " + setType + " dataset")

In [ ]:
# Download videos in synthetic dataset from YouTube
base_path = "/content/drive/MyDrive/Thesis/Dataset/Videos/"  
download_youtube_videos(train_videos, base_path, "Train")
download_youtube_videos(validation_videos, base_path, "Validation")
download_youtube_videos(test_videos, base_path, "Test")

In [ ]:
# Download videos in real-time dataset from YouTube
real_base_path = "/content/drive/MyDrive/Thesis/Dataset/"
download_youtube_videos(df_real_url, base_path, "Real")

**Convert input videos to frames**

In [ ]:
# Function to convert YouTube videos to frames at the given frame rate
def video_to_frames(ip_path, train_op_path, val_op_path, frames_rate, splitset = False):
  count = 1
  split_value = math.ceil(len(os.listdir(ip_path)) * 0.6)
  out_path = train_op_path
  for video_name in os.listdir(ip_path):
    if video_name.endswith(".mp4"):
      if count >= split_value and splitset == True:
        out_path = val_op_path
      (ffmpeg.input(ip_path+r"/"+video_name).filter('fps', fps=frames_rate).output(out_path+r"/"+video_name[:-4]+"Frame%d.jpg").run())
      count = count + 1

In [ ]:
#Convert video to frames for synthetic dataset
syn_input_file_path = r"/content/drive/MyDrive/Thesis/Dataset/Videos/Train"
frame_rate = 1
train_output_file_path = r"/content/drive/MyDrive/Thesis/Dataset/Videos/Train/TrainFrames"
val_output_file_path = r"/content/drive/MyDrive/Thesis/Dataset/Videos/Train/ValFrames"

video_to_frames(syn_input_file_path, train_output_file_path, val_output_file_path, frame_rate, splitset = True)

In [ ]:
#Convert video to frames for real scenario videos
real_ip_file_path = r"/content/drive/MyDrive/Thesis/Dataset/Real"
frame_rate = 1
real_op_path = r"/content/drive/MyDrive/Thesis/Dataset/Real/RealVideoFrames"
val_output_file_path = ""

video_to_frames(real_ip_file_path, real_op_path, val_output_file_path, frame_rate, splitset = False)

**Clone YOLOv5 model repository**

In [ ]:
%cd /content/drive/MyDrive/Thesis/Model
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

**Change YOLOv5 Data Configurations file**

In [ ]:
# Create own data.yml file with custom classes to be detected
config = {'path': '/content/drive/MyDrive/Thesis/Dataset/Videos/Train/',
         'train': '/content/drive/MyDrive/Thesis/Dataset/Videos/Train/TrainFrames/',
          'val': '/content/drive/MyDrive/Thesis/Dataset/Videos/Train/ValFrames/',
         'nc': 1,
         'names': ["Accident"]}
 
with open("/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml", "w") as file:
   yaml.dump(config, file, default_flow_style=False)

**Change YOLOv5s Model Configurations File**

In [ ]:
@register_line_cell_magic
def change_config_file(line,cell):
  with open(line,"w") as f:
    f.write(cell.format(**globals()))

In [ ]:
#Changed the number of classes. The architecture remains the same
%%change_config_file /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml

# Parameters
nc: 1  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

**Training the model**
<br> **Using Pre-trained Model:**

Hyperparameters:
Batch size = 64, Epochs=20, Image size = 640, Freeze Layers = None, Optimizer = SGD, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 640 --batch 64 --epochs 20 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights /content/drive/MyDrive/Thesis/Model/yolov5/yolov5s.pt --device 0 --cache --project /content/drive/MyDrive/Thesis/Results/res2

Hyperparameters:
Batch size = 64, Epochs=20, Image size = 640, Freeze Layers = 9, Optimizer = SGD, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 640 --batch 64 --epochs 20 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights /content/drive/MyDrive/Thesis/Model/yolov5/yolov5s.pt --device 0 --freeze 9 --project /content/drive/MyDrive/Thesis/Results/res1

Hyperparameters:
Batch size = 64, Epochs=20, Image size = 640, Freeze Layers = 10, Optimizer = SGD, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 640 --batch 64 --epochs 20 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights /content/drive/MyDrive/Thesis/Model/yolov5/yolov5s.pt --device 0 --freeze 10 --project /content/drive/MyDrive/Thesis/Results/res3

Hyperparameters:
Batch size = 64, Epochs=20, Image size = 640, Freeze Layers = None, Optimizer = Adam, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 640 --batch 64 --epochs 20 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights /content/drive/MyDrive/Thesis/Model/yolov5/yolov5s.pt --optimizer "Adam" --device 0 --project /content/drive/MyDrive/Thesis/Results/res5

Hyperparameters:
Batch size = 64, Epochs=20, Image size = 640, Freeze Layers = 9, Optimizer = Adam, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 640 --batch 64 --epochs 20 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights /content/drive/MyDrive/Thesis/Model/yolov5/yolov5s.pt --optimizer "Adam" --device 0 --freeze 9 --project /content/drive/MyDrive/Thesis/Results/res6

Hyperparameters:
Batch size = 128, Epochs=20, Image size = 640, Freeze Layers = None, Optimizer = SGD, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 640 --batch 128 --epochs 20 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights /content/drive/MyDrive/Thesis/Model/yolov5/yolov5s.pt --device 0 --project /content/drive/MyDrive/Thesis/Results/res4

Hyperparameters:
Batch size = 64, Epochs=20, Image size = 512, Freeze Layers = None, Optimizer = SGD, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 512 --batch 64 --epochs 20 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights /content/drive/MyDrive/Thesis/Model/yolov5/yolov5s.pt --device 0 --project /content/drive/MyDrive/Thesis/Results/res4

Hyperparameters:
Batch size = 64, Epochs=20, Image size = 512, Freeze Layers = 9, Optimizer = SGD, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 512 --batch 64 --epochs 20 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights /content/drive/MyDrive/Thesis/Model/yolov5/yolov5s.pt --device 0 --freeze 9 --project /content/drive/MyDrive/Thesis/Results/res7

Hyperparameters:
Batch size = 64, Epochs=100, Image size = 640, Freeze Layers = 9, Optimizer = SGD, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 640 --batch 64 --epochs 100 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights /content/drive/MyDrive/Thesis/Model/yolov5/yolov5s.pt --device 0 --freeze 9 --project /content/drive/MyDrive/Thesis/Results/res8

Hyperparameters:
Batch size = 64, Epochs=200, Image size = 640, Freeze Layers = 9, Optimizer = SGD, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 640 --batch 64 --epochs 200 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights /content/drive/MyDrive/Thesis/Model/yolov5/yolov5s.pt --device 0 --freeze 9 --project /content/drive/MyDrive/Thesis/Results/res9

**Training from scratch**

Hyperparameters:
Batch size = 64, Epochs=20, Image size = 640, Optimizer = SGD, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 640 --batch 64 --epochs 20 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights "" --device 0 --project /content/drive/MyDrive/Thesis/Results/res10

Hyperparameters:
Batch size = 64, Epochs=20, Image size = 640, Optimizer = Adam, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 640 --batch 64 --epochs 20 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights "" --optimizer "Adam" --device 0 --project /content/drive/MyDrive/Thesis/Results/res11

Hyperparameters:
Batch size = 64, Epochs=100, Image size = 640, Optimizer = SGD, Learning Rate=0.01

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 640 --batch 64 --epochs 100 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --weights "" --device 0 --project /content/drive/MyDrive/Thesis/Results/res12

**Fine-Tuning the best model**

**Change YOLOv5s Hyperparameter configuration file**

In [ ]:
# Change the learning rate in the hyperparameter config file. Rest of the hyperparameters remain the same.
%%change_config_file /content/drive/MyDrive/Thesis/Model/yolov5/data/hyps/custom_hyps.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for low-augmentation COCO training from scratch
# python train.py --batch 64 --cfg yolov5n6.yaml --weights '' --data coco.yaml --img 640 --epochs 300 --linear
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.001  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.01  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)


Hyperparameters:
Batch size = 64, Epochs=50, Image size = 640, Optimizer = SGD, Learning Rate=0.001

In [ ]:
!python /content/drive/MyDrive/Thesis/Model/yolov5/train.py --img 640 --batch 64 --epochs 50 --data "/content/drive/MyDrive/Thesis/Model/yolov5/data.yaml" --cfg /content/drive/MyDrive/Thesis/Model/yolov5/models/custom_yolov5s.yaml --hyp "/content/drive/MyDrive/Thesis/Model/yolov5/data/hyps/custom_hyps.yaml" --weights /content/drive/MyDrive/Thesis/Results/res1/exp3/best.pt --device 0 --project /content/drive/MyDrive/Thesis/Results/res13

**Video Summarization**

**Load Model with best weights**

In [ ]:
yolo_custom = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/Thesis/Results/res13/exp/weights/best.pt')  # local model

**Save details of each frame**

In [ ]:
# This will save frame number, duration, model prediction and the confidence score for each frame in the video in a csv file
def save_csv(save_directory, video_path):
  accident_pred = []
  accident_conf = []
  frame_position_list = []
  frame_duration_list = []
  frame_sec_list = []
  vidcap = cv2.VideoCapture(video_path)
  success,image = vidcap.read()
  while success:
    frame_index = int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))
    frame_sec = round(vidcap.get(cv2.CAP_PROP_POS_MSEC)/1000,2)
    frame_duration = str(datetime.timedelta(seconds=frame_sec))
    frame_position_list.append(frame_index)
    frame_duration_list.append(frame_duration)
    frame_sec_list.append(frame_sec)

    yolo_result = yolo_custom(image)
    if (yolo_result.pandas().xyxy[0].name == "Accident").any():
      accident_pred.append("1")
      accident_conf.append(yolo_result.pandas().xyxy[0].confidence.values[0])
    else:
      accident_pred.append("0")
      accident_conf.append(0)
    success,image = vidcap.read()
  video_details_df = pd.DataFrame({"Frame_position": frame_position_list,
                                   "Time_sec": frame_sec_list,
                                   "Time": frame_duration_list,
                                   "Prediction": accident_pred,
                                   "Confidence": accident_conf
                                   })
  video_details_df.to_csv(save_directory + "/VideoDetails.csv", index=False, header=True)

**Apply event based video summarization and generate short video clips**

In [ ]:
# This will generate short video clips of accidents
def generate_videos(save_directory, video_path, csv_path):
  csv_details = pd.read_csv(csv_path)
  accident_df = csv_details[csv_details["Prediction"] == 1]
  frame_pos_details = []
  start_frame = accident_df.Frame_position.values[0]
  end_frame = accident_df.Frame_position.values[0]

  vidcap = cv2.VideoCapture(video_path)
  ip_width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
  ip_height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

  accident_frames = accident_df.Frame_position.values.tolist()
  accident_sec = accident_df.Time_sec.values.tolist()
  font = cv2.FONT_HERSHEY_PLAIN
  acc_frame_count = 0
  counter = 1

  for i in range(len(accident_frames)):
    prev_frame = accident_frames[i-1] if i!=0 else accident_frames[0]
    if accident_frames[i] >= end_frame and accident_frames[i] - prev_frame <= int(fps):
      end_frame = accident_frames[i]
      acc_frame_count = acc_frame_count + 1
    else:
      if (start_frame - int(fps/2)) < 0:
        start_frame = 1
      else:
        start_frame = start_frame - int(fps/2)
      
      if end_frame + int(fps/2) > length:
        end_frame = length
      else:
        end_frame = end_frame + int(fps/2)

      if acc_frame_count > 1:
        start_dur = csv_details[csv_details["Frame_position"] == start_frame].Time_sec.values[0]
        end_dur = csv_details[csv_details["Frame_position"] == end_frame].Time_sec.values[0]
        frame_pos_details.append([start_dur,end_dur])
        acc_frame_count = 0
      
      start_frame = accident_frames[i]
  for frame in frame_pos_details:
    start_val , end_val = frame
    ffmpeg_extract_subclip(video_path, start_val, end_val, targetname= save_directory + f"/Summary{counter}.mp4")
    counter = counter+1

In [ ]:
save_path = "/content/drive/MyDrive/Thesis/Results/RealVideos/"
video_path_val = "/content/drive/MyDrive/Thesis/Dataset/Real/"

for file in video_path_val:
  if file.endswith(".mp4"):
    summary_op_save_path = save_path + file[:-4]
    summary_ip_video_path = video_path_val + "/" + file
    save_csv(summary_op_save_path, summary_ip_video_path)
    generate_videos(summary_op_save_path, summary_op_save_path,save_path+"/VideoDetails.csv")


**Evaluation Metrics**

In [ ]:
def plot_data(confusion_matrix):
  ax = sns.heatmap(confusion_matrix, annot=True, cmap='Purples', fmt= ".0f")
  ax.set_title('Confusion Matrix for Synthetic Video 1\n\n');
  ax.set_xlabel('\nPredicted Values')
  ax.set_ylabel('Actual Values ');

  ax.xaxis.set_ticklabels(['Background','Accident'])
  ax.yaxis.set_ticklabels(['Background','Accident'])

  plt.show()

In [ ]:
def evaluation(model_ground_truth_path, model_predictions_path):
  ground_truth_details = pd.read_csv(model_ground_truth_path)
  pred_details = pd.read_csv(model_predictions_path)

  confusion_matrix = metrics.confusion_matrix(ground_truth_details, pred_details)
  plot_data(confusion_matrix)

  accuracy = metrics.accuracy_score(ground_truth_details, pred_details)
  precision = metrics.precision_score(ground_truth_details, pred_details)
  recall = metrics.recall_score(ground_truth_details, pred_details)
  f1_score = metrics.f1_score(ground_truth_details, pred_details)


In [ ]:
for file in video_path_val:
  if file.endswith(".mp4"):
    gt_path = save_path + file[:-4] + "/GroundTruth.csv"
    pred_details = video_path_val + "/" + file
    evaluation(save_path, pred_details)
